In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install fuzzywuzzy[speedup]


In [ ]:
import pandas as pd
import re
#import pymorphy2
import math
import spacy
from collections import Counter
import numpy as np
import scipy.spatial
import collections
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Чтение данных
Сет распределен по нескольким файлам, поэтому необходимо их объединить.
Сет содержит данные о сложности вопроса по мнению спрашивающего и отвечающего и номер статьи, из которой взят ответ - нам это не нужно

In [ ]:
answers1 = pd.read_csv('S08_question_answer_pairs.txt', sep='\t')
answers2 = pd.read_csv('S09_question_answer_pairs.txt', sep='\t')
answers3 = pd.read_csv('S10_question_answer_pairs.txt', sep='\t', encoding='latin-1')

answers = pd.concat([answers1, answers2, answers3])
answers.drop(labels=['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile'], axis=1, inplace=True)
answers.head(20)

In [ ]:
print('Наличие null:', answers.isnull().values.any())
print('Наличие NaN:', answers.isna().values.any())

# Первичная обработка данных
## Обработка дубликатов и пустых значений

1. Сет содержит вопросы без ответа
2. Сет содержит повторяющиеся вопросы 

Дубликаты неоходимо удалить - они не несут дополнительной смысловой нагрузки, но увеличивают объем для обработки.
Пары с пустыми значениями (без ответа) не могут принеси пользы для модели + нельзя выполнить замену для пустых начени, поскольку необходимо знать правилный ответ. В результате остается только удалить такие вопросы.

Сначала удаляем вопросы без ответа, затем повторяющиеся вопросы

In [ ]:
print('Размер начального сета', answers.size)

#предполагается, что нам необходимо заполнение всех полей
not_null_answers = answers.dropna(axis=0, how='any')
print('Размер сета без пустых значений', not_null_answers.size)

# при удалении дупликатов оставляем только первое вхождение
no_duplicates_answers = not_null_answers.drop_duplicates(subset='Question', keep='first')
print('Размер сета без дубликатов', no_duplicates_answers.size)
no_duplicates_answers.head(20)


In [ ]:
print('Наличие null:', no_duplicates_answers.isnull().values.any())
print('Наличие NaN:', no_duplicates_answers.isna().values.any())

## Обработка притяжательных местоимений

1. В сете содержатся указания на одушевленный предмет статьи -his/her, he/she 
#####    (напр. Was his (Alessandro_Volta) 1800 paper written in French ?)
2. В сете содержатся указания на неодушевленный предмет статьи -this
#####    (напр. 	What connected the Akans to this (Ghana) Empire?)

Производится замена местоимений на тему, однако только в тех случаях, когда тема не содержится в вопросе и это местоимение является явным на неё указателем: 
##### (напр. "When did Lincoln begin his political career?").
В данном случае замена не производится.

In [ ]:
nlp = spacy.load('en_core_web_sm')

#no_duplicates_answers['Question'] =no_duplicates_answers['Question'].replace(regex=[r' his | her | this '], 
#                                                                           value=' замена ')
exp = no_duplicates_answers.copy()
exp['Theme column'] = ''
for index, row in exp.iterrows():
    row['ArticleTitle'] = row['ArticleTitle'].replace('_', ' ')
        
    title_words = list(map(lambda x: x.lower(),row['ArticleTitle'].split()))
    
    #question_words = re.findall(r"[a-zA-Z]+-[a-zA-Z]+|[a-zA-Z]+\'[a-zA-Z]+|[a-zA-Z]+", row['Question'])
    
    doc = nlp(row['Question'])
    infinitives = [token.lemma_ for token in doc]
    '''
    infinitives = []
    morph = pymorphy2.MorphAnalyzer()
    for word in question_words:
        word = word.lower()
        infinitives.append(morph.parse(word)[0].normal_form)        
    '''    
    
    infinitives = list(map(lambda x: x.lower(), infinitives))
    has_title_in_question = any(map(lambda x: x in list(map(lambda x: x.lower(), infinitives)), title_words))
    
    if (not has_title_in_question):      
        '''
        print(title_words)
        print(infinitives)
        print(row['ArticleTitle'], row['Question'])
        s = row['Question'].lower()        
        print(re.sub(r'( he ?)|( his )|( her )|( its )]', ' '+ row['ArticleTitle'] + ' ', ' ' + s + ' ').strip())
        ''' 
        s = row['Question'].lower()  
        row['Theme column'] = re.sub(r'( he )|( his )|( her )|( its )]', ' '+ row['ArticleTitle'] + ' ', ' ' + s + ' ').strip()
        
exp.head(20)

### Перенесем все неизмененные значения из столбца Question в Theme column. С последней и будем в дальнейшем работать

In [ ]:
for index, row in exp.iterrows():
    #print(row['Theme column'])
    row['Theme column'] = row['Theme column'].lower() if row['Theme column'] != '' else row['Question'].lower() 
    row['Theme column'] =   row['Theme column'].replace('?','').strip() 
exp.head(20)

# Перейдем к Токенизации для последующего TF-IDF и WB


In [ ]:
text = ' '.join(exp['Theme column'])
tokens = re.findall(r"[\w']+", text.lower())
print('\nTokens:\n\n', tokens[:500])


### Счетчик слов:


In [ ]:
term_freq_dict = Counter(tokens)
term_freq_tuple = [(key, value) for key, value in term_freq_dict.items()]
sorted(term_freq_tuple, key=lambda x: x[1], reverse=True)

In [ ]:
Q1, Q2, Q3 = np.percentile(list(term_freq_dict.values()), [25, 50, 75])
IQR = Q3 - Q1
lower_inner_fence = Q1 - (1.5 * IQR)
lower_outer_fence = Q1 - (3 * IQR)
upper_inner_fence = Q3 + (1.5 * IQR)
upper_outer_fence = Q3 + (3 * IQR)
print(f'''upper_outer_fence:  {upper_outer_fence}, 
upper_inner_fence:  {upper_inner_fence},
lower_inner_fence:  {lower_inner_fence},
lower_outer_fence"  {lower_outer_fence}''')

## Теперь немного подправим список стоп-слов (но только не значащие). 
### В нашем случае (в английском языке) — это напр. артикли.
### Было решено считать за стоп слова  все слова с частотой при токенизации >50.
#### Но с некоторыми исключениями: мы не удаляем уточняющие слова - what/where/e.t.c. (в финальн Более того, можно оставлять в списке только основную форму слова (напр. глагола become), поскольку данные в сете будут Лемматизированны.



#### Произведем изменение списка стоп-слов

In [ ]:
my_stop_words = nlp.Defaults.stop_words
print('\nИзначальный набор\n',len(my_stop_words))
#добавление отловленных слов из списка токенизированных, кроме when/where, language 
my_stop_words|= {'the','of','is','a', 'was','to','are','and','do','for','does','have','as',
                 'an','are','many','he','she','it','on','his','her','by','with','that','this','most','at','there','short','long',
                '\'s','.','?','!',',','&'}
#удаление лишних слов.  Удаляем только одну форму - become (см выше)
my_stop_words-= {'amount', 'another', 'anything', 'become'}
#'what',
#'when',
#'where',
#'whereafter',
#'which',
# 'who',
# 'whom',
#'whose',
#'why',
 

my_stop_words

### Удаляем стоп-слова из сета

In [ ]:
for index, row in exp.iterrows():
    new_row = ''
    for word in  row['Theme column'].split():
        if word not in my_stop_words:
            new_row += ' '+word
    row['Theme column']=new_row
exp.head(50)




# МЕШОК СЛОВ

In [ ]:
full_tokens = set(' '.join(tokens).split())
vectors = [[sentence.count(token) for token in tokens]
           for sentence in tokens]
print([(i, word) for i, word in enumerate(full_tokens)])
#vectors

In [ ]:
test_sentence = vectors[1] #choose any
for i, sentence in enumerate(tokens):
    print(sentence)
    print(scipy.spatial.distance.cosine(test_sentence, vectors[i]))
    

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = list('|'.join(text).split('|'))

vectorizer = CountVectorizer()
print( vectorizer.fit_transform(corpus).todense() )
print( vectorizer.vocabulary_ )

# Алгоритм Шинглов — поиск нечетких дубликатов текста


In [ ]:
def genshingle(source):
    import binascii
    shingleLen = 3 #длина шингла - 3--5--7
    out = [] 
    for i in range(len(source)-(shingleLen-1)):
        out.append (binascii.crc32(' '.join( [x for x in source[i:i+shingleLen]] ).encode('utf-8')))

    return out

def compaire(source1,source2):
    same = 0
    for i in range(len(source1)):
        if source1[i] in source2:
            same = same + 1

    return same*2/float(len(source1) + len(source2))*100

def test():
    text1 = exp.iloc[0]["Theme column"] # Текст 1 для сравнения - abraham lincoln sixteenth president united states
    text2 = 'lincoln sixteenth president' # Текст 2 для сравнения - обработанный
    text3 = 'was abraham linсoln the 16\'th president of Ameriсa' # Текст 3 для сравнения -не очищенный
    text4 = 'barak obama' # Текст 4 - совсем другой

    cmp1 = genshingle(text1)
    cmp2 = genshingle(text2)
    cmp3 = genshingle(text3)
    cmp4 = genshingle(text4)

    print ('\n'+text1)
    print (text2)


    print ('\n pretty near '+str(compaire(cmp1,cmp2)))
    
    print ('\n'+text1)
    print (text3)
    print ('\n Not pretty near '+str(compaire(cmp1,cmp3)))
    
    print ('\n'+text1)
    print (text4)
    print ('\n Not near almost '+str(compaire(cmp1,cmp4)))

# Start program
test()

# TF, IDF и TF-IDF

In [ ]:
def compute_tf(text):
    #Считаем частотность всех терминов во входном массиве с помощью 
    #метода Counter библиотеки collections, но уже в очищенной Theme column
    tf_text = collections.Counter(text)
    for i in tf_text:
        tf_text[i] = tf_text[i]/float(len(text))
    return tf_text
def compute_idf(word, corpus):
#на вход берется слово, для которого считаем IDF
#и корпус документов в виде списка списков слов
        #количество документов, где встречается искомый термин
        #считается как генератор списков
        return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))
def compute_idf_another(word, corpus):
    data = [Counter(i) for i in corpus if word in i]
    final_counter = Counter()
    for i in data:
        final_counter += i
    most_common_word = final_counter.most_common(1)[0][1]
    return math.log10(1 + (most_common_word/float(sum([1 for i in corpus if word in i]))))

def compute_tfidf(corpus):

    documents_list = []
    for text in corpus:
        tf_idf_dictionary = {}
        computed_tf = compute_tf(text)
        for word in computed_tf:
            tf_idf_dictionary[word] = computed_tf[word] * compute_idf(word, corpus)
        documents_list.append(tf_idf_dictionary)
    return documents_list

def compute_tfidf_another(corpus):

    documents_list = []
    for text in corpus:
        tf_idf_dictionary = {}
        computed_tf = compute_tf(text)
        for word in computed_tf:
            tf_idf_dictionary[word] = computed_tf[word] * compute_idf_another(word, corpus)
        documents_list.append(tf_idf_dictionary)
    return documents_list


In [ ]:
text = exp["Theme column"]
text_separated_by_word = list(map(lambda x: x.split(),'|'.join(text).split('|')))
tf=compute_tf(text)
#print(tf)
 

In [ ]:
print(compute_idf('president',text_separated_by_word))


In [ ]:
tfidf=compute_tfidf(text_separated_by_word)

print(tfidf)

In [ ]:

tfidf_another=compute_tfidf_another(text_separated_by_word)

 
print(tfidf_another)

# Косинусное расстояние

In [ ]:
#немного переделенные методы для tf-idf 
import operator


def tokenize(doc):
    words = [word.replace(',', '').lower() for word in doc.split()]
    return words


def build_terms(corpus):
    terms = {}
    current_index = 0
    for doc in corpus:
        for word in tokenize(doc):
            if word not in terms:
                terms[word] = current_index
                current_index += 1
    return terms


def tf(document, terms):
    words = tokenize(document)
    total_words = len(words)
    doc_counter = Counter(words)
    for word in doc_counter:
        # Можно и не делить, а оставить как есть, с частотой
        doc_counter[word] /= total_words
    tfs = [0 for _ in range(len(terms))]
    for term, index in terms.items():
        tfs[index] = doc_counter[term]
    return tfs


def _count_docs_with_word(word, docs):
    counter = 1
    for doc in docs:
        if word in doc:
            counter += 1
    return counter


def idf(documents, terms):
    idfs = [0 for _ in range(len(terms))]
    total_docs = len(documents)
    for word, index in terms.items():
        docs_with_word = _count_docs_with_word(word, documents)
        idf = 1 + math.log10(total_docs / docs_with_word)
        idfs[index] = idf
    return idfs


def _merge_td_idf(tf, idf, terms):
    return [tf[i] * idf[i] for i in range(len(terms))]


def build_tfidf(corpus, document, terms):
    doc_tf = tf(document, terms)
    doc_idf = idf(corpus, terms)
    return _merge_td_idf(doc_tf, doc_idf, terms)


def cosine_similarity(vec1, vec2):
    def dot_product2(v1, v2):
        return sum(map(operator.mul, v1, v2))

    def vector_cos5(v1, v2):
        prod = dot_product2(v1, v2)
        len1 = math.sqrt(dot_product2(v1, v1))
        len2 = math.sqrt(dot_product2(v2, v2))
        return prod / (len1 * len2)
         

    return vector_cos5(vec1, vec2)







In [ ]:
tf_idf_total = []
corpus = (tuple(text))[:100]
terms = build_terms(corpus)

for document in corpus:
    tf_idf_total.append(build_tfidf(corpus, document, terms))

#for doc_rating in tf_idf_total:
    #print(doc_rating)
print(terms.keys())
query = 'president'
print("QUERY:",query )
query_tfidf = build_tfidf(corpus, query, terms)
for index, document in enumerate(tf_idf_total):
    print("Similarity with DOC", index, "=", cosine_similarity(query_tfidf, document))

# Расстояние Ливенштейна

In [ ]:
exmpl = exp.iloc[0]["Theme column"]
print(exmpl)
print(fuzz.token_sort_ratio(exmpl, 'lincoln sixteenth president'))
print(fuzz.token_set_ratio(exmpl, 'lincoln sixteenth president '))


# Использование моделей для непосредственного поиска

In [ ]:
input_example = 'Did the election of 1880 was won by Lincoln?'

#clear input
clear_input_exmpl=''
new_row = ''

#Stop words

for word in  input_example.split():
   
    if word not in my_stop_words:
        new_row += ' '+word
        clear_input_exmpl=new_row.strip() .lower().replace('?','')

print(clear_input_exmpl) 


## Левенштейн

In [ ]:
levin_counter=0
levin_question=''
for question in text:
    if(fuzz.token_set_ratio(question, clear_input_exmpl)>=levin_counter):
        levin_counter=fuzz.token_set_ratio(question, clear_input_exmpl)
#       levin_counter=fuzz.token_sort_ratio(question, clear_input_exmpl)
        levin_question= question

        
print('it is most similar to:')
print(levin_question)
print('Levenshtein distance is:')
print(levin_counter)
print('Quesion And Answer are:')
exp.loc[exp['Theme column'] == levin_question, ['Question','Answer']]

#### Поиск возможных ответов с пониженным расстоянием Левинштейна

In [ ]:
possible_delta= 2
all_levenstain_distance = list((map(lambda x: fuzz.token_set_ratio(x, clear_input_exmpl),exp['Theme column'])))
                             #  >levin_counter-possible_delta,exp['Theme column']))

lev_possible_answers = list(map(lambda x: x>= levin_counter-possible_delta,all_levenstain_distance))
lev_possible_answers_with_distance = list(zip(exp['Theme column'][lev_possible_answers],
                                          exp['Answer'][lev_possible_answers],
                                          pd.Series(all_levenstain_distance)[lev_possible_answers]))
print('\nAlso possible questions are:')
lev_possible_answers_with_distance

## Шинглы

In [ ]:
shingle_counter=0
shingle_question=''
main_cmp = genshingle(clear_input_exmpl)
for question in text:
    tmp_cmp = genshingle(question)
    similatrity=compaire(main_cmp,tmp_cmp)
    if(similatrity>shingle_counter):
            shingle_counter=similatrity
            shingle_question=question
print('it is most similar to:')
print(shingle_question)
print('Shingle similarity procent is:')
print(shingle_counter)
print('Quesion And Answer are:')
exp.loc[exp['Theme column'] == shingle_question, ['Question','Answer']]

#### Поиск возможных ответов с пониженным показетелем схожести Шинглов

In [ ]:
possible_delta= 0.05
all_shingle_distance = list((map(lambda x: compaire(main_cmp,genshingle(x)),exp['Theme column'])))
                           

Sh_possible_answers = list(map(lambda x: x>= shingle_counter-possible_delta,all_shingle_distance))
Sh_possible_answers_with_distance = list(zip(exp['Theme column'][Sh_possible_answers],
                                          exp['Answer'][Sh_possible_answers],
                                          pd.Series(all_shingle_distance)[Sh_possible_answers]))
print('\nAlso possible questions are:')
Sh_possible_answers_with_distance

## TF-IDF

In [ ]:
print('See is it a low coefficient in these words?:')
print('\n for Levenstain')
print(pd.Series(tfidf)[lev_possible_answers].values[0])
print(pd.Series(tfidf_another)[lev_possible_answers].values[0])
###
print('\n for Shingles')
print(pd.Series(tfidf)[Sh_possible_answers].values[0])
print(pd.Series(tfidf_another)[Sh_possible_answers].values[0])

In [ ]:
clear_input_exmpl


# Проверка данных на тестовой выборке

## Очистка и обработка данных

In [ ]:
test_answers_1 = pd.read_csv('test_questions.csv', sep=';', encoding='latin-1')

test_answers = pd.concat([test_answers_1])

test_exp=test_answers.copy()
test_exp['Theme column'] = ''
#remove words
for index, row in test_exp.iterrows():
    row['ArticleTitle'] = row['ArticleTitle'].replace('_', ' ')
    row['Answer'] = row['Answer'].lower()    
    title_words = list(map(lambda x: x.lower(),row['ArticleTitle'].split()))
    
    
    doc = nlp(row['Question'])
    infinitives = [token.lemma_ for token in doc]
      
    infinitives = list(map(lambda x: x.lower(), infinitives))
    has_title_in_question = any(map(lambda x: x in list(map(lambda x: x.lower(), infinitives)), title_words))
    
    if (not has_title_in_question):      
        s = row['Question'].lower()  
        row['Theme column'] = re.sub(r'( he )|( his )|( her )|( its )]', ' '+ row['ArticleTitle'] + ' ', ' ' + s + ' ').strip()
#move all to Tc
for index, row in test_exp.iterrows():
    row['Theme column'] = row['Theme column'].lower() if row['Theme column'] != '' else row['Question'].lower() 
    row['Theme column'] =   row['Theme column'].replace('?','').strip() 
#Stop words
for index, row in test_exp.iterrows():
    new_row = ''
    for word in  row['Theme column'].split():
        if word not in my_stop_words:
            new_row += ' '+word
    row['Theme column']=new_row    
test_exp.head(20)

# Для каждого способа составим вероятный вопрос
### В столбцах содержатся вероятные вопросы

In [ ]:
test_exp['Fact'] = '' 
test_exp['Levin_question'] = '' #предикт вопроса по левинштейну
test_exp['Levin_answer'] = ''
test_exp['Shingles_question'] = '' #предикт вопроса по шинглам
test_exp['Shingles_answer'] = ''

for index, row in test_exp.iterrows():
    
    #levin
    levin_counter=0
    levin_question=''
    for question in text:
        if(fuzz.token_set_ratio(question, row['Theme column'])>=levin_counter):
            levin_counter=fuzz.token_set_ratio(question, row['Theme column'])
            levin_question= question
    row['Levin_question'] = str(exp.loc[exp['Theme column'] == levin_question, ['Question']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower()
    row['Levin_answer'] = str(exp.loc[exp['Theme column'] == levin_question, ['Answer']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower()
    
    # shingle
    shingle_counter=0
    shingle_question=''
    main_cmp = genshingle(row['Theme column'])
    for question in text:
        tmp_cmp = genshingle(question)
        similatrity=compaire(main_cmp,tmp_cmp)
        if(similatrity>shingle_counter):
            shingle_counter=similatrity
            shingle_question=question
    #print(shingle_question)      
    row['Shingles_question'] = str(exp.loc[exp['Theme column'] == shingle_question, ['Question']].values[0]).replace('[\'','').replace('\']','').lower()
    row['Shingles_answer'] = str(exp.loc[exp['Theme column'] == shingle_question, ['Answer']].values[0]).replace('[\'','').replace('\']','').lower()


## Поскольку cos-ое расстояние и список всех векторов требует неадекватно много времени:
#### Сравним алгоритмы со стандартным-реализованным решением


In [ ]:
import difflib

def similarity(s1, s2):
    normalized1 = s1.lower()
    normalized2 = s2.lower()
    matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
    return matcher.ratio()


In [ ]:
test_exp['difflib_question'] = '' #предикт вопроса по difflib
test_exp['difflib_answer'] = ''
difflib_counter=0
difflib_question=''
for index, row in test_exp.iterrows():
    for question in text:
        if(similarity(question, row['Theme column'])>=difflib_counter):
            difflib_counter=similarity(question, row['Theme column'])
            difflib_question= question
    row['difflib_question'] = str(exp.loc[exp['Theme column'] == difflib_question, ['Question']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower()
    row['difflib_answer'] = str(exp.loc[exp['Theme column'] == difflib_question, ['Answer']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower()


In [ ]:
test_exp.head(50)

In [ ]:
# 0 если ответы не сошлись, 1 если сошлись
test_exp['Levin_count'] = ''
test_exp['Shingle_count'] = ''
test_exp['difflib_count'] = ''
for index, row in test_exp.iterrows():
    
    if(row['Levin_answer']==row['Answer']):
        row['Levin_count']=1
    else:
        row['Levin_count']=0

    if(row['Shingles_answer']==row['Answer']):
        row['Shingle_count']=1
    else:
        row['Shingle_count']=0
    
    if(row['difflib_answer']==row['Answer']):
        row['difflib_count']=1
    else:
        row['difflib_count']=0
    row['Fact']=str(exp.loc[exp['Question'] == 'Did Lincoln start his political career in 1832?', ['Answer']]).split()[2].lower()

In [ ]:
test_exp.head(50)

In [ ]:
print('Левинштейн точность')
test_exp['Levin_count'].sum()/test_exp['Levin_count'].count()

In [ ]:
print('Шинглы точность')
test_exp['Shingle_count'].sum()/test_exp['Shingle_count'].count()

In [ ]:
print('difflib точность')
test_exp['difflib_count'].sum()/test_exp['difflib_count'].count()

In [ ]:
test_exp.to_csv('result.csv', sep=';')